## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from datetime import date
from dateutil.relativedelta import relativedelta
import yfinance as yf
from numpy.linalg import inv
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

plt.rcParams["figure.figsize"] = (20,20)

## Building a class backtest the tangency portfolio weightings

In [2]:
class tangency_backtester:
    def __init__(self, Return, equity, r, train_test_ratio, Capital):
        
        # Store the variables
        self.Return = Return
        self.equity = equity
        self.r = r
        self.Capital = Capital
        
        # Split the set
        n = round(len(Return) * train_test_ratio)
        self.Return_train = self.Return.iloc[:n]
        self.Return_test = self.Return.iloc[n:]
        self.equity_train = self.equity.iloc[:n]
        self.equity_test = self.equity.iloc[n:]
        
        # Get the required matrices/ vectors
        self.mu_vector = 252 * np.array(self.Return_train.mean()).T
        self.one_vector = np.ones(len(self.Return_train.columns)).T
        self.Cov_inverse = inv(252 * np.array(self.Return_train.cov()))
        self.Cov = 252 * np.array(self.Return_train.cov())
    
        # Calculate the necessary variables
        self.c_1_1 = self.one_vector.T.dot(self.Cov_inverse).dot(self.one_vector)
        self.c_1_mu = self.one_vector.T.dot(self.Cov_inverse).dot(self.mu_vector)
        self.c_mu_mu = self.mu_vector.T.dot(self.Cov_inverse).dot(self.mu_vector)
     
    # Get the gamma (sharpe), mean, SD and weightings of the tengency porfolio
    def fit(self):
        self.gamma = abs(np.sqrt(self.c_mu_mu - 2 * self.r * self.c_1_mu + (self.r**2) * self.c_1_1))
        self.mu_t = (self.c_mu_mu - self.r * self.c_1_mu) / (self.c_1_mu - self.r * self.c_1_1)
        self.sigma_t = self.gamma / (self.c_1_mu - self.r * self.c_1_1)
        self.weightings_t = self.Cov_inverse.dot(self.mu_vector - self.r * self.one_vector) / (self.c_1_mu - self.r * self.c_1_1)
    
    # Plot the tengancy portfolio of train set (log scale)
    def plot_train(self):
        portfolio_equity = pd.Series(data=self.equity_train.dot(self.weightings_t), index=self.equity_train.index)
        portfolio_return = np.log(portfolio_equity/ portfolio_equity.shift())
        portfolio_return.dropna(inplace=True)
        plt.plot(portfolio_return.cumsum())
        plt.show()
        plt.plot(portfolio_equity)
        plt.show()
        print(portfolio_equity[-1]/ portfolio_equity[0] - 1)
        
    # Plot and analysis the tengancy portfolio of test set
    def plot_test(self):        
        portfolio_equity = pd.Series(data=self.equity_test.dot(self.weightings_t), index=self.equity_test.index)
        plt.plot(portfolio_equity)
        plt.show()

        
    # Plot the combined equity curve and log return of train and test sets
    def plot_train_test(self):
        portfolio_equity_train = pd.Series(data=self.equity_train.dot(self.weightings_t), index=self.equity_train.index)
        portfolio_equity_test = pd.Series(data=self.equity_test.dot(self.weightings_t), index=self.equity_test.index)
        if portfolio_equity_train.min() <= 0:
            Min = abs(portfolio_equity_train.min()) + 2
            portfolio_equity_train += Min
            portfolio_equity_test += Min
        plt.plot(portfolio_equity_train, c="b", label="Train")
        plt.plot(portfolio_equity_test, c="orange", label="Test")
        plt.title("Equity curve")
        plt.xlabel("Date")
        plt.ylabel("Equity")
        plt.legend()
        plt.show()
        portfolio_return_train = pd.Series(np.log(portfolio_equity_train / portfolio_equity_train.shift()), index=portfolio_equity_train.index)
        portfolio_return_test = pd.Series(np.log(portfolio_equity_test / portfolio_equity_test.shift()), index=portfolio_equity_test.index)
        portfolio_return_train.dropna(inplace=True)
        portfolio_return_test.dropna(inplace=True)
        portfolio_return_test[0] += portfolio_return_train.cumsum()[-1]
        plt.plot(portfolio_return_train.cumsum(), c="b", label="Train")
        plt.plot(portfolio_return_test.cumsum(), c="orange", label="Test")
        plt.title("Cumulative return (log scale)")
        plt.xlabel("Date")
        plt.ylabel("Return")
        plt.legend()
        plt.show()
    

    # Return the log return series of the test set
    def get_test(self):
        initial_price = self.equity_test.iloc[0]
        holding = []
        for i, price in enumerate(initial_price):
            holding.append(self.Capital * self.weightings_t[i] / price)
        holding = np.array(holding)
        portfolio_equity_test = pd.Series(data=self.equity_test.dot(holding), index=self.equity_test.index)
        portfolio_return_test = pd.Series(np.log(portfolio_equity_test / portfolio_equity_test.shift()), index=portfolio_equity_test.index)
        portfolio_return_test.dropna(inplace=True)
        return portfolio_return_test, portfolio_equity_test
    
